In [ ]:
#Import libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import regex as re# module is an alternative implementation of regular expressions that
#extends and enhances the capabilities of the standard re module.


In [ ]:
#Understanding and preprocessing the dataset


In [ ]:
'''
that takes a file path as input, reads the content of the file, and then splits the text into sentences
 based on common sentence-ending punctuation marks ('.', '?', and '!').
 The function returns a list of sentences.
'''
def file_to_sentence_list(file_path):
	with open(file_path, 'r') as file:
		text = file.read()

# Splitting the text into sentences using
# delimiters like '.', '?', and '!'

	sentences = [sentence.strip() for sentence in re.split(
		r'(?<=[.!?])\s+', text) if sentence.strip()]

	return sentences

In [ ]:
'''
 file_to_sentence_list to read the content of the file and tokenize it into sentences.
 After that, you're using the Keras Tokenizer class to tokenize the text data into words and
 determine the total number of unique words in the text.
'''
file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path)

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_data)
total_words = len(tokenizer.word_index) + 1

In [ ]:
# Create input sequences
input_sequences = []
for line in text_data:
	token_list = tokenizer.texts_to_sequences([line])[0]
	for i in range(1, len(token_list)):
		n_gram_sequence = token_list[:i+1]
		input_sequences.append(n_gram_sequence)

In [ ]:
# Pad sequences and split into predictors and label
max_sequence_len = max([len(seq) for seq in input_sequences])
input_sequences = np.array(pad_sequences(
	input_sequences, maxlen=max_sequence_len, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# Convert target data to one-hot encoding
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

In [ ]:
# Define the model
model = Sequential()
model.add(Embedding(total_words, 10,input_length=max_sequence_len-1))
model.add(LSTM(128))
model.add(Dense(total_words, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X, y, epochs=50, verbose=1)


Epoch 1/50
51/51 [==============================] - 7s 80ms/step - loss: 6.2105 - accuracy: 0.0510
Epoch 2/50
51/51 [==============================] - 3s 50ms/step - loss: 5.7871 - accuracy: 0.0565
Epoch 3/50
51/51 [==============================] - 3s 50ms/step - loss: 5.7281 - accuracy: 0.0522
Epoch 4/50
51/51 [==============================] - 3s 51ms/step - loss: 5.7020 - accuracy: 0.0498
Epoch 5/50
51/51 [==============================] - 4s 72ms/step - loss: 5.6675 - accuracy: 0.0528
Epoch 6/50
51/51 [==============================] - 4s 74ms/step - loss: 5.6275 - accuracy: 0.0534
Epoch 7/50
51/51 [==============================] - 3s 51ms/step - loss: 5.5793 - accuracy: 0.0565
Epoch 8/50
51/51 [==============================] - 3s 51ms/step - loss: 5.5246 - accuracy: 0.0590
Epoch 9/50
51/51 [==============================] - 3s 51ms/step - loss: 5.4718 - accuracy: 0.0694
Epoch 10/50
51/51 [==============================] - 4s 82ms/step - loss: 5.4096 - accuracy: 0.0682
Epoch 11/

In [ ]:
# Generate next word predictions
seed_text = "Pizza"
next_words = 5

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences(
		[token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted_probs = model.predict(token_list)
	predicted_word = tokenizer.index_word[np.argmax(predicted_probs)]
	seed_text += " " + predicted_word

print("Next predicted words:", seed_text)


1/1 [==============================] - 0s 48ms/step
Next predicted words: Pizza has become a symbol of
